In [0]:
#https://github.com/Shivampanwar/Bert-text-classification/blob/master/bert_language_model_with_sequence_classification.ipynb
import os
import numpy as np
import pandas as pd
import re
from tqdm import tqdm
import warnings
#warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
import torch
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from tqdm import  tqdm_notebook

In [0]:
DIRETORIO = '/content/drive/My Drive/BERT/ANTON'
data = pd.read_csv(os.path.join(DIRETORIO,'final.csv'))

In [34]:
data = data.drop('Unnamed: 0', axis=1)
data.head()

,review_rate,review_body
0,10,"Pois bem...as fotos dos pratos, bebidas e doce..."
1,30,Espero que utilizem essa avaliação para rever ...
2,20,"Fomos comer a sobremesa as 20h40, sentamos, pe..."
3,10,Pedimos o cardápio e ao chamar o atendente ped...
4,50,"Fui com algumas amigas em uma segunda-feira, d..."


In [35]:
data.shape

(7672, 2)

In [0]:
#primeiro fine tunar no teste, dps testar no treino e dai testar no teste

In [0]:
data_reviews_treino = data['review_body'][:6000]

In [38]:
data_reviews_treino.head()

0    Pois bem...as fotos dos pratos, bebidas e doce...
1    Espero que utilizem essa avaliação para rever ...
2    Fomos comer a sobremesa as 20h40, sentamos, pe...
3    Pedimos o cardápio e ao chamar o atendente ped...
4    Fui com algumas amigas em uma segunda-feira, d...
Name: review_body, dtype: object

In [0]:
data_reviews_treino = data_reviews_treino.str.lower()

In [40]:
data_reviews_treino.head()

0    pois bem...as fotos dos pratos, bebidas e doce...
1    espero que utilizem essa avaliação para rever ...
2    fomos comer a sobremesa as 20h40, sentamos, pe...
3    pedimos o cardápio e ao chamar o atendente ped...
4    fui com algumas amigas em uma segunda-feira, d...
Name: review_body, dtype: object

In [0]:
tqdm.pandas()
changed_text = data_reviews_treino.apply(lambda x:x+"\n"+"\n")

In [42]:
changed_text.head()

0    pois bem...as fotos dos pratos, bebidas e doce...
1    espero que utilizem essa avaliação para rever ...
2    fomos comer a sobremesa as 20h40, sentamos, pe...
3    pedimos o cardápio e ao chamar o atendente ped...
4    fui com algumas amigas em uma segunda-feira, d...
Name: review_body, dtype: object

In [43]:
open(os.path.join(DIRETORIO,'data_lm_anton.txt'), "w").write(''.join(changed_text))


1299402

In [44]:
!pip install pytorch_transformers
from pytorch_transformers import *

In [45]:
!git clone https://github.com/nlpyang/pytorch-transformers

Cloning into 'pytorch-transformers'...
remote: Enumerating objects: 6469, done.
remote: Total 6469 (delta 0), reused 0 (delta 0), pack-reused 6469
Receiving objects: 100% (6469/6469), 3.63 MiB | 2.83 MiB/s, done.
Resolving deltas: 100% (4633/4633), done.


In [16]:
%cd pytorch-transformers/examples/lm_finetuning

/content/pytorch-transformers/examples/lm_finetuning


In [0]:
import shutil

In [18]:
src = DIRETORIO+"/data_lm_anton.txt" ##copying newly created data to the same finetuned folder.
dst = os.getcwd()
shutil.copy(src, dst)

'/content/pytorch-transformers/examples/lm_finetuning/data_lm_anton.txt'

In [19]:
os.listdir(os.getcwd())

['data_lm_anton.txt',
 'simple_lm_finetuning.py',
 'pregenerate_training_data.py',
 'finetune_on_pregenerated.py',
 'README.md']

In [20]:
!pip install transformers

     |████████████████████████████████| 573kB 2.8MB/s 
     |████████████████████████████████| 3.7MB 8.3MB/s 


In [21]:
!python3 pregenerate_training_data.py --train_corpus data_lm_anton.txt --output_dir training/ --epochs_to_generate 25 --max_seq_len 256

2020-04-22 01:33:24.126469: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
Downloading: 100% 921/921 [00:00<00:00, 744kB/s]
Downloading: 100% 210k/210k [00:00<00:00, 380kB/s] 
Downloading: 100% 2.00/2.00 [00:00<00:00, 1.46kB/s]
Downloading: 100% 112/112 [00:00<00:00, 84.6kB/s]
Downloading: 100% 43.0/43.0 [00:00<00:00, 30.3kB/s]
Loading Dataset: 12000 lines [00:03, 3123.35 lines/s]
Epoch:   0% 0/25 [00:00<?, ?it/s]
Document:   0% 0/6000 [00:00<?, ?it/s]
Document:   9% 531/6000 [00:00<00:01, 5305.20it/s]
Document:  16% 987/6000 [00:00<00:00, 5052.27it/s]
Document:  25% 1505/6000 [00:00<00:00, 5089.64it/s]
Document:  33% 2001/6000 [00:00<00:00, 5047.20it/s]
Document:  41% 2473/6000 [00:00<00:00, 4941.14it/s]
Document:  49% 2954/6000 [00:00<00:00, 4898.13it/s]
Document:  58% 3477/6000 [00:00<00:00, 4991.38it/s]
Document:  66% 3983/6000 [00:00<00:00, 5007.48it/s]
Document:  75% 4487/6000 [00:00<00:00, 5015.24it/s]
Docume

In [22]:
os.listdir(os.getcwd())

['data_lm_anton.txt',
 'simple_lm_finetuning.py',
 'training',
 'pregenerate_training_data.py',
 'finetune_on_pregenerated.py',
 'README.md']

In [48]:
!python3 finetune_on_pregenerated.py --pregenerated_data training/ --do_lower_case --train_batch_size 2  --output_dir finetuned_lm/ --epochs 2

2020-04-22 02:18:40.161028: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-04-22 02:18:41,925: device: cuda n_gpu: 1, distributed training: False, 16-bits training: False
2020-04-22 02:18:42,696: loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/neuralmind/bert-base-portuguese-cased/config.json from cache at /root/.cache/torch/transformers/aac3429673975db22f5d8a9202bc6a8983145bbd621577c9f2f62bee7fd02934.67c09492031a448a8963c917df31f30f95a403a40b4e44dda4b115a5d6c59f40
2020-04-22 02:18:42,696: Model config BertConfig {
  "_num_labels": 2,
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bad_words_ids": null,
  "bos_token_id": null,
  "decoder_start_token_id": null,
  "directionality": "bidi",
  "do_sample": false,
  "early_stopping": false,
  "eos_token_id": null,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,


In [0]:
#problema no automodel. Usando o multi por enquanto. Linha 239
np.random.seed(56)

In [0]:
#data_reviews_treino.review_body = data_reviews_treino.review_body.str.lower()
sentences = data_reviews_treino.values

# We need to add special tokens at the beginning and end of each sentence for BERT to work properly
sentences = ["[CLS] " + sentence + " [SEP]" for sentence in sentences]
labels = data['review_rate'][:6000].values

In [0]:
model_path="/content/pytorch-transformers/examples/lm_finetuning/finetuned_lm/"

In [54]:
#=====================VAI PRO 39===============================================

%%time
tokenizer = BertTokenizer.from_pretrained('/content/pytorch-transformers/examples/lm_finetuning/finetuned_lm')
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
print ("Tokenize the first sentence:")
print (tokenized_texts[0])

Tokenize the first sentence:
['[CLS]', 'pois', 'bem', '.', '.', '.', 'as', 'fotos', 'dos', 'pratos', ',', 'bebidas', 'e', 'doces', 'são', 'maravilhos', '##as', '.', '.', '.', 'daquela', '##s', '[UNK]', 'v', '##c', 'se', 'emp', '##ol', '##ga', 'e', 'diz', ':', 'vale', 'a', 'pena', 'conhecer', '.', '.', 'porém', 'tudo', 'acaba', 'quando', 'v', '##c', 'entra', '.', '.', '.', 'inf', '##elizmente', '.', 'cheg', '##ue', '##i', 'com', 'meu', 'marido', 'e', 'sent', '##amos', '.', '.', 'mesa', 'suj', '##a', '.', '.', 'mas', 'ficam', '##os', 'aguarda', '##ndo', 'o', 'atendimento', '.', '.', '.', '5', 'min', '.', ',', '10', 'min', '.', ',', '(', 'o', 'atende', '##nte', 'chegou', 'na', 'mesa', 'ao', 'lado', ',', 'tirou', 'o', 'pedido', 'e', 'nada', 'de', 'nos', 'atender', '.', 'outro', 'atende', '##nte', 'serviu', 'a', 'outra', 'mesa', 'ao', 'lado', '.', '.', '13', 'minutos', '.', 'aper', '##tam', '##os', 'o', 'botão', 'pra', 'acion', '##ar', 'o', 'atende', '##nte', '(', 'aquele', '[UNK]', 'fica',

In [57]:
input_ids=[]
for i in tqdm_notebook(range(len(tokenized_texts))):
    input_ids.append(tokenizer.convert_tokens_to_ids(tokenized_texts[i]))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [0]:
MAX_LEN = 256
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

In [0]:
#Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)

# Use train_test_split to split our data into train and validation sets for training

train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels,random_state=56, test_size=0.2)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids,random_state=56, test_size=0.2)

#Convert all of our data into torch tensors, the required datatype for our model

train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

batch_size = 16

# Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop, 
# with an iterator the entire dataset does not need to be loaded into memory

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [60]:
## Define model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device ='cpu'
model = BertForSequenceClassification.from_pretrained('/content/pytorch-transformers/examples/lm_finetuning/finetuned_lm',num_labels=2)
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [0]:
lr = 2e-5
max_grad_norm = 1.0
num_total_steps = 1000
num_warmup_steps = 100
warmup_proportion = float(num_warmup_steps) / float(num_total_steps)  # 0.1


### In PyTorch-Transformers, optimizer and schedules are splitted and instantiated like this:
optimizer = AdamW(model.parameters(), lr=lr, correct_bias=False)  # To reproduce BertAdam specific behavior set correct_bias=False
scheduler = WarmupLinearSchedule(optimizer, warmup_steps=num_warmup_steps, t_total=num_total_steps)  # PyTorch scheduler

In [63]:
total_step = len(train_dataloader)

# Store our loss and accuracy for plotting
train_loss_set = []

# Number of training epochs (authors recommend between 2 and 4)
epochs = 2

# trange is a tqdm wrapper around the normal python range
for epoch in tqdm_notebook(range(epochs)):
  
  

    # Training
    # Set our model to training mode (as opposed to evaluation mode)
    model.train()

    # Tracking variables
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0

    # Train the data for one epoch
    for i, batch in enumerate(train_dataloader):
      # Add batch to GPU
      batch = tuple(t.to(device) for t in batch)
      # Unpack the inputs from our dataloader
      b_input_ids, b_input_mask, b_labels = batch
      # Forward pass
      outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
      loss = outputs[0]
      train_loss_set.append(loss.item())    
      # Backward pass
      loss.backward()
      # Update parameters and take a step using the computed gradient
      optimizer.step()
      scheduler.step()
      optimizer.zero_grad()
      if (i) % 50 == 0:
        print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, epochs, i+1, total_step, loss.item()))

torch.save(model.state_dict(), DIRETORIO+'/model_with_language_model.ckpt')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # Remove the CWD from sys.path while we load stuff.


RuntimeError: ignored

In [0]:
torch.save(model.state_dict(), DIRETORIO+'/model_with_language_model.ckpt')

In [0]:
# Test the model
with torch.no_grad():
    correct = 0
    total = 0
    for i, batch in enumerate(validation_dataloader):
      batch = tuple(t.to(device) for t in batch)
      # Unpack the inputs from our dataloader
      b_input_ids, b_input_mask, b_labels = batch
      # Forward pass
      outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
      # print (outputs)
      prediction = torch.argmax(outputs[0],dim=1)
      total += b_labels.size(0)
      correct+=(prediction==b_labels).sum().item()

In [0]:
print('Test Accuracy of the model on val data is: {} %'.format(100 * correct / total))

In [0]:
#PARA RODAR:
#tira o --bert-model da chamada dele, comenta a parte do argumento
#importa o tranformers e chama ele no código, igual ao github do negocio 
#=================================ou=====================================
#roda tudo no internacional